In [5]:
import pandas as pd
import numpy as np

# Load the dataset
df = pd.read_csv('wppool_growth_data_sample_20k.csv')
df.head()




,user_id,install_date,last_active_date,subscription_type,country,total_sessions,page_views,download_clicks,activation_status,days_active,pro_upgrade_date,plan_type,monthly_revenue,churned
0,1,6/29/2023,7/12/2023,Free,UK,3,15,1,1,13,NaN,NaN,0,1
1,2,4/10/2023,7/25/2023,Free,India,133,665,0,1,106,NaN,NaN,0,0
2,3,10/25/2023,12/7/2023,Free,USA,53,106,0,1,43,NaN,NaN,0,0
3,4,8/26/2023,11/9/2023,Pro,Canada,242,242,0,1,75,11/9/2023,Basic,49,0
4,5,5/14/2023,11/22/2023,Free,UK,12,48,0,1,192,NaN,NaN,0,0


In [2]:
# Step 1: Percentage of users who upgraded from Free to Pro
total_users = df['user_id'].nunique()
upgraded_users = df[df['subscription_type'] == 'Pro']['user_id'].nunique()
upgrade_percentage = (upgraded_users / total_users) * 100
print(f"Percentage of users who upgraded from Free to Pro: {upgrade_percentage:.2f}%")



Percentage of users who upgraded from Free to Pro: 20.14%


In [3]:
# Step 2: Total monthly revenue from Pro users
total_revenue = df[df['subscription_type'] == 'Pro']['monthly_revenue'].sum()
print(f"Total monthly revenue from Pro users: ${total_revenue:,.2f}")



Total monthly revenue from Pro users: $235,481.00


In [4]:
# Step 3: Which Pro plan contributes the most revenue?
revenue_by_plan = df[df['subscription_type'] == 'Pro'].groupby('plan_type')['monthly_revenue'].sum()
print("\nRevenue contribution by Pro plan:")
print(revenue_by_plan)




Revenue contribution by Pro plan:
plan_type
Basic         80339
Enterprise    78764
Standard      76378
Name: monthly_revenue, dtype: int64


In [6]:
# Step 4: Analyze how long it takes for Free users to upgrade based on country and engagement level

# Filter Free users who upgraded
free_to_pro_users = df[(df['subscription_type'] == 'Pro') & (df['pro_upgrade_date'].notnull())].copy()

# Calculate the time taken to upgrade (in days)
free_to_pro_users.loc[:, 'upgrade_time'] = (
    pd.to_datetime(free_to_pro_users['pro_upgrade_date']) - 
    pd.to_datetime(free_to_pro_users['install_date'])
).dt.days

# Group by country and calculate the average upgrade time
upgrade_time_by_country = free_to_pro_users.groupby('country')['upgrade_time'].mean().sort_values()

# Print results
print("\nAverage time to upgrade (in days) by country:")
print(upgrade_time_by_country)





Average time to upgrade (in days) by country:
country
UK           87.899654
USA          90.242215
France       91.164311
Germany      91.525510
India        92.578151
Canada       93.571184
Australia    93.683919
Name: upgrade_time, dtype: float64


In [9]:


# Group by engagement level (total_sessions) and calculate average upgrade time
free_to_pro_users = free_to_pro_users.copy()  # Ensure we're working with a separate copy

# Create engagement level categories safely using .loc
free_to_pro_users.loc[:, 'engagement_level'] = pd.cut(
    free_to_pro_users['total_sessions'], 
    bins=[0, 10, 50, 100, np.inf], 
    labels=['Low', 'Medium', 'High', 'Very High']
)


# Group by engagement level and calculate the average upgrade time
upgrade_time_by_engagement = free_to_pro_users.groupby(
    'engagement_level', observed=False  # Explicitly setting observed=False
)['upgrade_time'].mean().sort_values()


# Print results
print("\nAverage time to upgrade (in days) by engagement level:")
print(upgrade_time_by_engagement)



Average time to upgrade (in days) by engagement level:
engagement_level
High         88.775988
Very High    91.706561
Medium       93.716578
Low          99.818182
Name: upgrade_time, dtype: float64
